In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import RepeatedStratifiedKFold


In [3]:
df = pd.read_csv('combined_features.csv',index_col=0)
df

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread,forward_return,output
time,,,,,,,,,,,
2020-11-17 17:00:00,0.949281,0.911684,0.908723,0.910738,1.028341,0.291573,0.913092,1.458798,8.522631,-0.147019,-1
2020-11-18 10:00:00,0.949618,0.912712,0.910101,0.912506,1.154188,0.337144,1.264965,2.788685,7.923738,-0.195590,-1
2020-11-18 11:00:00,0.949919,0.914357,0.912081,0.915681,1.443286,0.416619,1.582164,2.204730,7.326875,-0.288873,-1
2020-11-18 12:00:00,0.950429,0.917156,0.914294,0.919506,1.567864,0.517426,2.864586,11.746214,6.784229,-0.396706,-1
2020-11-18 13:00:00,0.950870,0.920886,0.916310,0.922577,1.556466,0.605077,3.047215,13.277380,6.601599,-0.430397,-1
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28 13:00:00,0.948541,0.837141,0.838733,0.836955,0.914318,-0.267499,-1.990000,0.492807,14.850000,0.071380,1
2022-10-28 14:00:00,0.948539,0.836728,0.838468,0.837909,0.968452,-0.254640,0.225000,9.559848,14.535000,0.054696,1
2022-10-28 15:00:00,0.948534,0.837559,0.838659,0.838728,0.973852,-0.245484,-0.322900,15.006942,14.582900,0.050257,1


In [4]:
# Reference https://blog.quantinsti.com/pairs-trading-basics/
interval = 24
threshold = 0.5
#df with spread
def strategy_labeling(df, interval, threshold):
    df["rolling_mean"] = df['spread'].rolling(interval).mean()
    df["rolling_std"] = df['spread'].rolling(interval).std()
    df.dropna(inplace=True)
    df["upper_threshold"] = threshold*df["rolling_std"]
    df["lower_threshold"] = - threshold*df["rolling_std"]
    df["z-score"] = (df['spread'] - df["rolling_mean"]) / df["rolling_std"]
    ### 1 is go short (sell es buy wec), 0 is hold, -1 is go long (sell wec buy es) ... I think
    # spr["label"] = 1 if spr[spr["z-score"] > spr["upper_threshold"]] else 0 if spr[spr["z-score"] < spr["lower_threshold"]] else -1
    df['label'] = df.apply(lambda x: 1 if x["z-score"] > x["upper_threshold"] else 0 if x["z-score"] < x["lower_threshold"] else -1, axis=1)
    return df

df = strategy_labeling(df, interval, threshold)
df

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread,forward_return,output,rolling_mean,rolling_std,upper_threshold,lower_threshold,z-score,label
time,,,,,,,,,,,,,,,,,
2020-11-20 16:00:00,0.952244,0.917706,0.916947,0.917443,0.977181,0.174480,-0.203764,-11.104774,7.602887,-0.124698,-1,7.388430,0.463642,0.231821,-0.231821,0.462549,1
2020-11-20 17:00:00,0.952364,0.917155,0.916049,0.915671,0.801230,0.120071,-0.503915,-12.365107,8.058714,-0.057567,-1,7.369100,0.422110,0.211055,-0.211055,1.633730,1
2020-11-23 10:00:00,0.952331,0.915942,0.916294,0.915487,0.877458,0.145069,-0.543661,-16.069909,7.697344,-0.029412,-1,7.359667,0.411574,0.205787,-0.205787,0.820451,1
2020-11-23 11:00:00,0.952405,0.915439,0.915780,0.915434,0.861316,0.128449,-0.671073,-33.293554,7.725438,0.051591,1,7.376274,0.418182,0.209091,-0.209091,0.834959,1
2020-11-23 12:00:00,0.952635,0.914917,0.915832,0.915208,0.801367,0.113063,-0.278762,-25.000367,7.736724,0.123113,1,7.415961,0.404526,0.202263,-0.202263,0.792934,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28 13:00:00,0.948541,0.837141,0.838733,0.836955,0.914318,-0.267499,-1.990000,0.492807,14.850000,0.071380,1,14.122750,0.381199,0.190600,-0.190600,1.907796,1
2022-10-28 14:00:00,0.948539,0.836728,0.838468,0.837909,0.968452,-0.254640,0.225000,9.559848,14.535000,0.054696,1,14.155875,0.381035,0.190517,-0.190517,0.994988,1
2022-10-28 15:00:00,0.948534,0.837559,0.838659,0.838728,0.973852,-0.245484,-0.322900,15.006942,14.582900,0.050257,1,14.186412,0.384805,0.192402,-0.192402,1.030360,1


In [5]:
split = round(0.8*len(df))
train, test = df[:split],df[split:]
x_train = train[['vwap','SMA(5)','SMA(10)','12dayEWM','rsi','MACD','mom','mfi','spread']]
y_train = train[['output']]

x_test = test[['vwap','SMA(5)','SMA(10)','12dayEWM','rsi','MACD','mom','mfi','spread']]
y_test = test[['output']]
y_pred = test[['label']]

In [6]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.28      0.05      0.09       291
           0       0.21      0.46      0.29       164
           1       0.77      0.87      0.81       331

    accuracy                           0.48       786
   macro avg       0.42      0.46      0.40       786
weighted avg       0.47      0.48      0.44       786

